# 📘 PyTorch Datasets & Transforms — Building a Custom Dataset and Composing Transforms

In this notebook, we will:

- 🔹 Build a **custom Dataset** class that behaves like a Python iterable
- 🔹 Add a **transform pipeline** using callable classes
- 🔹 **Compose multiple transforms** and apply them in sequence
- 🔹 Verify behavior with indexing `[]`, `len()`, and iteration


---

## 🔹 1️⃣ What is a Dataset?

A PyTorch **Dataset** wraps your data and exposes two key methods:

- `__len__(self)` → returns the **number of samples**
- `__getitem__(self, idx)` → returns the **sample at index** $\,i = \text{idx}$

We will implement a dataset with **$N = 100$** samples:

- Feature tensor $\color{#1f77b4}{x} \in \mathbb{R}^{N\times 1}$
- Target tensor $\color{#d62728}{y} \in \mathbb{R}^{N\times 1}$

A single sample is a **tuple**:  

$$
\text{sample}[i] = (\, \color{#1f77b4}{x_i},\; \color{#d62728}{y_i}\,)
$$

The dataset behaves like a list/iterable: you can use `len(dataset)`, `dataset[i]`, and `for s in dataset: ...`

---


In [1]:
# ===============================
# 1) Imports & setup
# ===============================
import torch
from torch.utils.data import Dataset

# For reproducibility when randomness is used (e.g., noise)
torch.manual_seed(0)


## 🏗️ 2️⃣ Build a Custom Dataset Class

We will create a class `ToySet(Dataset)` that:
- constructs feature/target tensors in `__init__`
- stores the sample count in `self.length`
- implements `__len__` to return the number of samples
- implements `__getitem__` to return a tuple `(x_i, y_i)`
- optionally applies a **transform** if provided

$$
\color{#1f77b4}{x_i} = x[i], \quad
\color{#d62728}{y_i} = y[i]
$$


In [2]:
# ============================================
# 2) Custom Dataset: ToySet
# ============================================
class ToySet(Dataset):
    def __init__(self, length=100, transform=None, device='cpu', dtype=torch.float32):
        """Create a toy regression dataset with a simple relation y = 2x + 1 + noise.

        Args:
            length (int): number of samples N (will be cast to int).
            transform (callable or None): optional callable applied to each (x, y) on access.
            device (str or torch.device): where to store tensors ('cpu' or 'cuda').
            dtype (torch.dtype): e.g., torch.float32.
        """
        # ---- Configuration ----
        self.length = int(length)                  # total number of samples
        self.transform = transform                 # optional transform pipeline
        self.device = torch.device(device)         # device handle

        # ---- Create features x in [0, 1] ----
        x_vals = torch.linspace(0, 1, self.length, dtype=dtype)  # evenly spaced grid

        # ---- Add small Gaussian noise for realism ----
        noise = 0.02 * torch.randn(self.length, dtype=dtype)     # zero-mean noise

        # ---- Define targets: y = 2x + 1 + noise ----
        y_vals = 2.0 * x_vals + 1.0 + noise

        # ---- Store as (N, 1) column vectors on chosen device ----
        self.x = x_vals.unsqueeze(1).to(self.device)  # shape (N, 1)
        self.y = y_vals.unsqueeze(1).to(self.device)  # shape (N, 1)

    def __len__(self):
        """Return number of samples so len(dataset) works and
        DataLoader knows the epoch length."""
        return self.length

    def __getitem__(self, idx):
        """Return a single sample (x_i, y_i). If a transform is provided,
        apply it before returning."""
        # 1) Select i-th row from x and y
        xi = self.x[idx]     # tensor of shape (1,)
        yi = self.y[idx]     # tensor of shape (1,)

        # 2) Pack into a tuple
        sample = (xi, yi)

        # 3) If a transform pipeline is configured, apply it here
        if self.transform is not None:
            sample = self.transform(sample)  # callable(sample) -> sample

        # 4) Return the (possibly transformed) sample
        return sample


## 🔍 3️⃣ Create a Dataset Instance & Inspect Behavior

- `len(ds)` returns the number of samples $N$  
- `ds[i]` returns the tuple $(\color{#1f77b4}{x_i}, \color{#d62728}{y_i})$  
- The dataset is **iterable**, so `for sample in ds:` works

We'll create a dataset without any transform first.


In [3]:
# ============================================
# 3) Create a dataset instance and inspect behavior
# ============================================
# Create dataset with no transform
ds_plain = ToySet(length=100)

# len(ds) -> calls __len__
print("len(ds_plain) ->", len(ds_plain))  # expect 100

# Index one item -> calls __getitem__
x0, y0 = ds_plain[0]
print("First sample shapes:", x0.shape, y0.shape)  # both (1,)
print("First sample values:", x0.item(), y0.item())

# Iterate first three items
print("\nFirst three samples:")
for i in range(3):
    xi, yi = ds_plain[i]
    print(f"[{i}] x={xi.item():.4f}, y={yi.item():.4f}")


len(ds_plain) -> 100
First sample shapes: torch.Size([1]) torch.Size([1])
First sample values: 0.0 0.9774832129478455

First three samples:
[0] x=0.0000, y=0.9775
[1] x=0.0101, y=0.9972
[2] x=0.0202, y=1.0354


---

## 🔧 4️⃣ Building a Transform (Callable Class)

Transforms are **callable objects** that take a sample `(x, y)` and return a (possibly) **modified** `(x, y)`.

We’ll create a transform class `AddMultiply` that:
- **adds** a constant to $\color{#1f77b4}{x}$ → $x' = x + \text{add\_x}$
- **multiplies** $\color{#d62728}{y}$ → $y' = (\text{mul\_y})\, y$

$$
\color{#1f77b4}{x'} = x + \text{add\_x}, \qquad
\color{#d62728}{y'} = (\text{mul\_y})\,y
$$

We can apply this transform **directly to a sample**, or pass it into the dataset so it’s applied **automatically** when indexing.


In [4]:
# ============================================
# 3) Transform class: AddMultiply
# ============================================
class AddMultiply:
    def __init__(self, add_x=0.0, mul_y=1.0):
        """Store hyperparameters for add/multiply operations."""
        self.add_x = float(add_x)
        self.mul_y = float(mul_y)

    def __call__(self, sample):
        """Make the object callable; transform interface: (x, y) -> (x', y')."""
        x, y = sample
        x = x + self.add_x   # add bias to feature
        y = y * self.mul_y   # scale target
        return (x, y)

# Demo: apply transform directly to a single sample
tform = AddMultiply(add_x=1.0, mul_y=2.0)
x0, y0 = ds_plain[0]
x0_t, y0_t = tform((x0, y0))

print('Original (x0, y0):', x0.item(), y0.item())
print('Transformed (x0, y0):', x0_t.item(), y0_t.item())


Original (x0, y0): 0.0 0.9774832129478455
Transformed (x0, y0): 1.0 1.954966425895691


---

## 🧰 5️⃣ Applying Transform Automatically (via Dataset Constructor)

We pass the transform instance to the dataset’s constructor:

$$
\text{ToySet}(*,\; \text{transform}=\mathcal{T})
$$

Now, each time we access `ds[i]`, the dataset will **apply** the transform before returning the sample.


In [5]:
# ============================================
# 4) Pass transform to the Dataset for automatic application
# ============================================
# Create dataset that applies AddMultiply automatically on access
ds_auto = ToySet(length=100, transform=AddMultiply(add_x=1.0, mul_y=2.0))

x0_auto, y0_auto = ds_auto[0]  # __getitem__ applies transform
print("Auto-transformed first sample:", x0_auto.item(), y0_auto.item())


Auto-transformed first sample: 1.0 2.0385007858276367


---

## 🧱 6️⃣ Composing Multiple Transforms (Sequentially)

Sometimes we want to **chain** several transforms. We can define a simple `Compose` class that
applies each transform **in order**:

$$
\mathcal{T}_{\text{compose}} = \mathcal{T}_2 \circ \mathcal{T}_1, \quad
\text{sample}' = \mathcal{T}_2(\mathcal{T}_1(\text{sample}))
$$

We’ll also create a `Mult` transform that multiplies **both** $x$ and $y$ by a constant:

$$
x' = (\text{mul})\,x, \qquad y' = (\text{mul})\,y
$$


In [6]:
# ============================================
# 5) Compose + an additional transform (Mult)
# ============================================
class Compose:
    def __init__(self, transforms):
        """Run a list of transforms sequentially."""
        self.transforms = list(transforms)

    def __call__(self, sample):
        for t in self.transforms:
            sample = t(sample)
        return sample

class Mult:
    def __init__(self, mul=1.0):
        """Multiply BOTH x and y by 'mul'."""
        self.mul = float(mul)

    def __call__(self, sample):
        x, y = sample
        return (x * self.mul, y * self.mul)

# Build a transform pipeline:
#   1) AddMultiply: (x+1, y*2)
#   2) Mult: multiply both by 0.5
pipeline = Compose([
    AddMultiply(add_x=1.0, mul_y=2.0),
    Mult(mul=0.5)
])

# Apply pipeline directly to a raw sample
x0_c, y0_c = pipeline(ds_plain[0])
print("Compose on sample[0]:", x0_c.item(), y0_c.item())

# Or pass pipeline into the dataset for automatic application
ds_comp = ToySet(length=100, transform=pipeline)
x1_c, y1_c = ds_comp[1]
print("Compose via dataset[1]:", x1_c.item(), y1_c.item())


Compose on sample[0]: 0.5 0.9774832129478455
Compose via dataset[1]: 0.5050504803657532 1.0545985698699951


---

## 🔄 7️⃣ Iterating Over the Dataset (First 3 Samples)

The Dataset is an **iterable**, so we can loop over it directly.  
Below, we loop over the **transformed** dataset and print the first three samples:

$$
\text{for } i \in \{0,1,2\}:\; (x_i, y_i) = \text{ds}[i]
$$


In [7]:
# ============================================
# 6) Iterate and preview the first three transformed samples
# ============================================
print('First 3 samples from composed-transform dataset:')
for i in range(3):
    xi, yi = ds_comp[i]  # triggers __getitem__ -> pipeline(sample)
    print(f'[{i}] x={xi.item():.4f}, y={yi.item():.4f}')

First 3 samples from composed-transform dataset:
[0] x=0.5000, y=0.9869
[1] x=0.5051, y=1.0546
[2] x=0.5101, y=1.0212


---

## ✅ Summary

- We built a custom Dataset that returns tuples $(\color{#1f77b4}{x_i}, \color{#d62728}{y_i})$ and behaves like a Python iterable.
- We created **callable transform classes** to modify samples.
- We showed **two ways** to apply transforms:
  1) **Directly** to returned samples, and  
  2) **Automatically** by passing a transform to the Dataset constructor.
- We implemented a simple **Compose** to chain multiple transforms in sequence.

📌 **Key idea:** The Dataset’s `__getitem__` is the perfect place to plug in transforms, ensuring every access returns **consistently processed** data.
